In [1]:
from datascience import *
%matplotlib inline
path_data = '../../../assets/data/'
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import math
from scipy import stats
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=np.VisibleDeprecationWarning)

# TKN Deficit Examples

A TKN is in deficit if the total vault balance is less than or equal to the staked amount, after accounting for the exit fee. It is necessary to separate the vault balance into the trading liquidity (b), and non-trading liquidity (c) components; b + c is the vault balance of TKN:

![](https://lh3.googleusercontent.com/fwaVj4wuMls4oRYlMxofIl4QMf8FFMyXMWVHdC_ZoryMa2xVBUB0rHQA22hn-jIrb1jTMhHO0U1qNbCFV415lCobHOLgxawTv10cqOkHcmCERYGjITjT9FkKayXQVzFCrU6tCNNe)

where e is the TKN balance of the staking ledger, and n is the withdrawal fee (e.g. 0.0025, or 0.25%). So long as the above expression is True, the protocol is in an effective deficit (i.e. the quantity of TKN is insufficient to reimburse the totality of all user stakes). In this state, it is possible that some users will receive partial reimbursement in BNT. However, the protocol will also attempt to reimburse users entirely in TKN using the reverse of the repricing strategy described for the surplus case - essentially to recapture TKN from the external markets. The following paragraphs will explore the three different situations that cover the discrete behaviors of the withdrawal algorithm during a deficit state of TKN.

As before, the potential for abuse is the most important component of the decision tree. The same thresholds, hlim and hmax, defined previously are used again here; however, the hmax calculation is modified to account for the reversed direction:

![](https://lh3.googleusercontent.com/yCN3qagE1MLNA24RqemiPLcPOD1_9fcR2wW0ZnlY2I_sBd_pebcdEy3veURQTeeV8HFk4iIKGkqyKILvHb8peoelQNN7ddnXBWHOtWFSyrkUTC2OXaJx9vwziVRKIju_dGilKM9t)

![](https://lh4.googleusercontent.com/UCJCVi2wQ_VqVFIXKdxvQG-2kMc5m2i7ZHSnP_zgUhfqbJntBTS7Xhrqy3HHumDOLS92VSrZyEh_i35wdnaq-RFYL9X2VTGm9xkcM_4NTuTgQsL1lzhben77dEL3lnIQ965oXJYu)

The hlim and hmax terms are used in exactly the same fashion as before; they define the maximum value for x that is allowable without creating a possible abuse incentive. Therefore, the user's withdrawal amount (x) must be less than both hlim and hmax:

![](https://lh3.googleusercontent.com/F9tBEhoDYsFtiDmCwTKoKjesPyCtxswCVOrZp7US5cwbsHHoVlQ6g2A35RckcliVov-u34yXsT_h7dvsGwzeYnFbV1vVt5xao81akBT4m2OmqwMkXMIerV52ED-VQaWwIfKxpJR1)

![](https://lh4.googleusercontent.com/7JE-5sMEyWoyQ26R8MQEvlRvYwimCBKSMxE7-F7ebYkJNNOpyx1UiKc35w0tNQ5MQST33UvOnaFn7vqkPy6izLMUzY09kxy0QQOIh3mosjcZPtY_HwCWNyscUa8KAWsPLI9u0qwG)

To examine this further, consider the following case: the staking ledger reports a staked balance of 2,000 TKN (e), the vault balance of TKN is 1,800 TKN (b+c), the TKN trading pool has 1,000 TKN (b) and 1,000 BNT (a) in available liquidity. Then, a user confirms a withdrawal for 100 TKN (x).

![](https://lh3.googleusercontent.com/iOlPrYHkJT1GOBBUJL4895W2FxpjaqOl_3fIb6MxKEve1ODqvG9dBs1qYqA4M1MHHJPa234qj84BJE74oJznd_TM3DVeqtWTvHQjjSH6IuHPaXP34KBJOx4JzBvFWl3ta8FFGEpD)

In this case, hlim evaluates to 888.888889 TKN, and hmax evaluates to 1138.958507 TKN. As the user's withdrawal amount (x) is less than both of these values, there must be sufficient non-trading TKN in the vault to support the withdrawal, and there is no financial incentive to create the withdrawal for the purpose of back running. Therefore the protocol can attempt to recover a small amount of TKN from the outside markets.

The withdrawal algorithm outputs are the same, although the sign is effectively reversed in P and R, relative to the surplus calculations. To maintain unsigned arithmetic, the following descriptions are provided:

P, BNT quantity to add (mint)  to the available trading liquidity.

Q, BNT ownership renounced by the protocol (i.e. removed from the staking ledger).

R, TKN quantity to remove from the available trading liquidity (and send to the user).

S, TKN quantity to remove from the vault (and send to the user).

T, BNT quantity minted for, and sent to the user as compensation.

U, TKN removed from the external protection wallet and sent to the user as compensation.

The method for calculating each of these outputs is dependent on the state of the system. The case where there is a TKN deficit, and the user's withdrawal satisfies the hlim and hmax checks, the outputs are calculated as follows:

![](https://lh4.googleusercontent.com/znIGMCx6gVdsqwBtR-a5XNyT2yW7shnMLEnzAOFm2CYVsZFzTuqVf_Gw-hGywfdIeSan8pYPZtOKn440ZiT_boh90-8LwSsblO6VddBjK3xVGmv26FzE0N8somZAR4EWHqObfLJ9)

![](https://lh4.googleusercontent.com/HWQ0ayRkFLt_pm4Ay5MpZLe7DYPTzZdmzJYzaP092rD1h3aGWVjAt6nBa8-F_MtSCvV8LqcWiz33YL2BSAUxdx-7ujK4MY5735a2K7LPo1bDi3SvVVttPkLN2UAJNruM4hO3cGl7)

![](https://lh4.googleusercontent.com/NJGFShc6h3aNK8zjEjc8W0M5rCSbi920lF81jhwkYETuxZ0kh2EdZrvZ-LXOgQ8QzTV8UMAJYLRZgA0c5mLUQMnhvsENKb3Eh42z7wgjKC4_Tzy0aO-Y6N7R9F_xn0sppiaXVRMX)

![](https://lh4.googleusercontent.com/VnDLyoZjmykCqhIK60Urz0lzHeCbptb-Gab8jCChueX5GyPZ0yPVLrrjxSh7Xr0Gk0XravkuXwvZYyi5iM7LOXFIUZEWC82eMUYhLB_Uhc1NCAG7LxQ0oqP9rUTfgKx2usit82Si)

![](https://lh3.googleusercontent.com/x6eOFRCThilA9PFv4iFVnEjQA16pH2tKlG3AYOD7dZ5qzE0nuUXS3E90D2un_N5akN53LlsslCPkb7GWYdvY7Jb82r5HtPd6t3kWodZPUpRuGu4JMK6YObIAfStbYoF6OJMXeO_Z)

![](https://lh6.googleusercontent.com/aAXwI_wmKVCPiQn63qQmK_-4Fh6H8SQqyngO5Jj8DgsPRzJQUuS0TmUFPOHo_zUnZwIu_UprDHxxdJ1qjzV34rjB8dtOCwKhMVtNc5p7acnT9uJMlI6FFf5dYaJPiI-3D2ifO5sF)

![](https://lh6.googleusercontent.com/lMF452wchGgnH8Df1KjH9CAaWiZYk5ex7T87-pV_vfyEFKIjwF9Nj0ipmPa1pZkk6_96I7rgAy6jyb9Qf10o6o8_TvWeg_y4dJ01z_k1PYCXv46YPpecW7nYe4FVBEASEM_aqu9M)

![](https://lh3.googleusercontent.com/QJ3lXBYj-DU2-dRJHuA-cvFRl52c4PL5hXcVe3QgvrmCY3dtoCSiygKOH9sbv7ClWKVomneMyf0T5m8_Hr75mBHiJnrc8zWzuaPEjhtc4__Sddr8D1XbHHNzwIKAXSV7uOCes-rM)

Note the opposite behavior of the protocol, compared to the surplus case. During a deficit, the algorithm reacts by quoting a higher price for TKN. The same considerations applied in the previous argument also apply here; if and when the pool returns to balance (defined as state of the pool before the withdrawal was executed), the relative deficit of TKN compared to the staked balance will remain constant, (ignoring vortex fee effects). The number of BNT minted to the trading liquidity, and the amount of TKN taken from the trading liquidity, is determined such that a hypothetically perfect arbitrage trade would return the deficit to its pre-withdrawal value. As before, there is no specific requirement for the implied arbitrage opportunity to close immediately, or even eventually.

The method is abusable if the withdrawal value (x) is greater than hlim or hmax. To examine a case where these abusability thresholds are exceeded, consider a scenario nearly identical to that presented above, but where the TKN balance of the staking ledger (e) is increased to 2,200 TKN. Therefore, the vault balance of TKN remains at 1,800 TKN (b+c), the TKN trading pool has 1,000 TKN (b) and 1,000 BNT (a) in available liquidity, and the user confirms a withdrawal for 100 TKN (x).

![](https://lh3.googleusercontent.com/h27ObZ02GdVoLTyarcGZB-4-ZhbBkyuVhPPNA-V9dqfgwAyEWZojVKuQnf7CTL-oFGUhhjTYWXL858BzUn1cN2I59nhUc5V2lu4g5Oh6EfHW-Hwm-Xlj2NojEpkOtu83vMHG4Ny7)

For this scenario, hlim evaluates to 977.777778 TKN, and hmax evaluates to 87.855051 TKN. As the user's withdrawal is still 100 TKN, the hlim test still passes, but the hmax test fails.

Therefore, the protocol's repricing strategy is potentially abusable. Similar to what was discussed in the surplus sections, the withdrawal algorithm defaults to another set of operations, and a bifurcation occurs depending on the sufficiency of the non-trading TKN balance of the vault.

![](https://lh3.googleusercontent.com/s3YSc9aMnlO4Rc5q0AyEfJnuO8PMwFRg8zULospVfPRIq-n5G1mdOQZmyK2s8dpJ9BWQDYGVXEg9vJqH4oYhD49SxxdZOPc7R0YsfSYgFkd89UM5dzI3nI2q96B1OcRCEQMZGWvE)

In this case the inequality is True; the user's withdrawal can be processed without recourse to reducing the trading liquidity. The outputs of the algorithm are calculated as follows:

![](https://lh4.googleusercontent.com/lJf6kRUn2_IW9grxaUiUMQo8suVq1erzjWOGY8o6huBTp_vXCUpUvSt415N8FYnj15zXdStYy1YYZ3_RZVGJvkfB_wj1Z_tRsdOczABQj7cnE2X3o3WUwm1r6I5L-e2HAxF1LNy4)

![](https://lh5.googleusercontent.com/4mufywW-XWRfF8aVDjZr2Jrr1sMWmBTGrgac2VFob4P-ZVPeIzkCFAkKbP7aS_7M_tvnXmb0zpBM900vCovVh6eO1WzBvQhdvXGNXCL6WGCXT9_25A3ykGZX22vzNfIJ51R5dbti)

![](https://lh4.googleusercontent.com/X6sBpEapvzU3MYooLiiqY4oDwUuTFoet8sQi8UinQGYhZ_zcwPg5Qef1ZgBQuEp0w8lM7HLmvori_sP6o6M2G4nkzz32dYm7AJATMjE_n4ArM_Of9kD59NCqIHl04qXSkHuOV5HE)

![](https://lh5.googleusercontent.com/GPohc_Q-Jhivy7wQFcxk5fawXF5WBPYZ57FI9EViltrTIT0GyYNKublF6pQHH4eY6zObVzuhWs0SjQN4UQ7SHb4R3LEl8kS4FL36NJs9KzYbXA4bGbv8PuMApQ4_zfCMQq9kR_ta)

![](https://lh5.googleusercontent.com/KKxgN54Un7JW-MkMN1vkQHYdv9jOuh-mWOAjugdmaFNjZ2Cm8ttJhP1nf_UUc5IN4x5MMBlhdRl_smjQNfplXhH_f2KuPfyi950weKkV-PrqJuusT6JoIay7_9qn3fVqJZqvvXYt)

![](https://lh4.googleusercontent.com/09bKRp73aJU-WjjAoeO8HVtoCCAMjqGluaU4k3aOh2XmpZ64sOAHxK-PsdV_IZFMZTB8eH4_zZrMaGD2Mdy9g5VVzUac1446VKYET6OOn0xuNkGXro4YO-IlGxLDYMIQ_ZtvNGen)

![](https://lh5.googleusercontent.com/VdwVskv_vrsEOfOuOUYCHCg8-2eVUXvaJ32jH_p8fkyqqTvKXjossnTOWwFKcI4bNIKhQaVrJ_S7y_wkv_GHANmMeRKsoJmILphtj-lBjKzvSYs5PmJ0d-UYZFbD4mHzxrzTvPUd)

![](https://lh3.googleusercontent.com/e3Na9PjHW0bryGV0_dEHXz4OEZSklBudRg6QcfPBndHeOTw5EStxl1iSlESjTKNj5IPy6iz6PVZ900E6SAli8gt2mYs9-LUwNCaE4o_zXC1nf_2P4uR6cCIzQ-hYY78sZYpN6NZI)

The most intuitive way to interpret this result is to add the TKN and BNT amounts together. In this case, the BNT/TKN rate is precisely 1:1, which makes it easy. The S and T outputs represent an amount of TKN, and BNT tokens sent to the user, respectively. Since these tokens have the same value, their sum should represent the total value the user was expecting, save for the exit fee. In this case, the sum is precisely 99.75, which is what we expect. These BNT tokens received in reimbursement are minted at withdrawal, and do not originate from inside the protocol. While this example uses a 1:1 valuation of BNT:TKN, the output T handles any BNT valuation.

Lastly, the case wherein the hlim or hmax test fails, and where it is impossible to refund the user without affecting the trading liquidity, should be considered. For this example, let the BNT trading liquidity (a) be 1,800 BNT, let the TKN trading liquidity (b) be 1,800 TKN, let the vault balance of TKN (b+c) be 1,850 TKN, and let the balance of TKN on the staking ledger (e) be 2,200 TKN, and assume the user confirms a withdrawal for 100 TKN (x).

![](https://lh5.googleusercontent.com/rx_HJGGThcMuh5aBxEHpRI5ruyKenEhlHJlbXXt-Ln1skySANEXxWT7pjG-OeHsOCj5ad4BBYrRo4JtWTaI1_EDZUgFILDFgU3wklyCa0HOGKJDOVUNeTybobT5-nrkND_obofUy)

For this scenario, hlim evaluates to 59.45945946 TKN, and hmax evaluates to 203.670372 TKN. As the user's withdrawal is still 100 TKN, the hlim test fails, but the hmax test passes. The repricing method is therefore abusable. As before, the protocol proceeds to check if the available non-trading liquidity is sufficient to cover the withdrawal.

![](https://lh3.googleusercontent.com/s3YSc9aMnlO4Rc5q0AyEfJnuO8PMwFRg8zULospVfPRIq-n5G1mdOQZmyK2s8dpJ9BWQDYGVXEg9vJqH4oYhD49SxxdZOPc7R0YsfSYgFkd89UM5dzI3nI2q96B1OcRCEQMZGWvE)

In this case the inequality is False. Therefore, it is necessary to reduce the available trading liquidity. A third series of methods is required to calculate each of the algorithm outputs:

![](https://lh4.googleusercontent.com/1Eakl_RY6JQl2nvgrs2YVGea-3fCXHht1NHszDKLRxduNAziJuwCZc2O_drDDbFmM5slMaJwSu96P4JyBNDHkC9iJgtu9fG6fGWBiMBVJ0TCmescBSvTDZm2TcZie_lAyVjuQxr9)

![](https://lh3.googleusercontent.com/zQklSqEfeyWOa9tnqs-aHt5Fb-iVtoMBsaxQ2hvBvjnv8Gwnl2W_PKLMJZHgm3jxno3-0wyht8cDe75kvYniHhkts9kHJ-zcHdfc23AeIgqDFVxyyr_l8iqru2l0U8Pzhdzax5tw)

![](https://lh6.googleusercontent.com/juuPdYND-J11ZqEWeSCpctbGIc_QbucrSh15rroZQyvxUGibb6WwN1PzFxysJE1Qt30-E9n8_Xb7aadoQWB-ieGpKR5149VuY1gc8ZCa3gq0SmLIVjGwJbCHXDXK-yPVgeA28osZ)

![](https://lh5.googleusercontent.com/tdHhCCLTz1ibvplS46nq5BvW_DG6kCtdVH1UKBD-iEGw_xtj73l_sIdKBc872o7mIcgZVraFj1G4lnKk_OCovd2D7g8w-EfiubrB0yO16-eocMSTJGWfnsm-z7f7pLQCwH1sC-Mp)

![](https://lh3.googleusercontent.com/G1U0nhyYfIi1alNH2Xn5xnCQ31fO87eV9I3Bzp7VStVA8Mh4tz163SQT9VtEAomkNgCtIf8kArOE4-rEt9ZnWppF4c4gaG4hJqIj2egMz0JfOgIF6fkUQY4z6TfgI-L4PicqdHPe)

![](https://lh6.googleusercontent.com/GWYSKur3NJDdQqqxI_mwL0xYb_wDpcwruep2cVoz4n3mOiSlbarm6XWlZciRVr7Roxz5DjARC-x55eeWSRzMn91rwbLnzyupaZg_x-WpUJOcTPrkGTLkUAeVKszuK54oqZJB6KSQ)

![](https://lh4.googleusercontent.com/LyhxGHZFLrjLfcgjBDMyL95EhSe0SKBuQ2FvmqMvQPGqPV00_dD9uYHaAgjhLk6jcJ1MKq-bpDj-Jn530X2bDdMnvVhBJi5tbcVVKYRKH-yzNOBvKyULp8NPDRTdGniW6D7LhBV4)

![](https://lh4.googleusercontent.com/sVJLAm_tQJptrisrvtu7l4_3nL06kLxa1lTD4k1Ud_LzQuyFzHP4j3h9imWWzlvw1eHHz1Q6GwpZbDQNbSgBU4LQHKszoJoK2XDbjL8Uof8fnYQHR5C_tta1ft0fPdaJOArLJEO_)

As with its surplus counterpart, the outcome of this scenario is that the protocol gives up what non-trading TKN was left in the vault, and then takes the remaining balance from the trading liquidity. As this results in a formal protocol withdrawal of BNT, the Q output is populated, and is later transformed into an equivalent amount of bnBNT tokens. Both the BNT and the bnBNT are burned, and the quantity of BNT on the staking ledger is adjusted down appropriately.